In [ ]:
# train_test/svm_train_test.py
import os
import sys
import pandas as pd
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, average_precision_score
from sklearn.preprocessing import OneHotEncoder, label_binarize
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import HashingVectorizer

核心库：
pandas/numpy：数据处理与数值计算。
scikit-learn：提供 SVM 模型、数据划分、评估指标等工具。
scipy：稀疏矩阵操作（csr_matrix）与.mat文件加载。
HashingVectorizer：高基数特征哈希编码，避免维度爆炸。

In [ ]:
# 直接硬编码项目根目录
PROJECT_ROOT = "E:/Finance_fraud/Finance_fraud"

# 验证项目根目录
if not os.path.exists(PROJECT_ROOT):
    raise ValueError(f"项目根目录不存在: {PROJECT_ROOT}")

# 将项目根目录添加到系统路径
sys.path.append(PROJECT_ROOT)
print(f"已添加项目根目录到路径: {PROJECT_ROOT}")

# 定义数据目录
DATADIR = os.path.join(PROJECT_ROOT, "data")
print(f"数据目录: {DATADIR}")

# 验证 models 目录存在
models_dir = os.path.join(PROJECT_ROOT, "models")
if not os.path.exists(models_dir):
    raise FileNotFoundError(f"models 目录不存在: {models_dir}")

# 导入模型
from models.svm_model import get_svm_model
print(f"成功导入 get_svm_model 函数")

Finance_fraud/                 # 项目根目录
├── data/                      # 数据集存放目录
│   ├── S-FFSD.csv             # 金融欺诈数据集
│   ├── Amazon.mat             # Amazon评论数据集
│   └── YelpChi.mat            # Yelp餐厅评论数据集
├── models/                    # 模型定义目录
│   └── svm_model.py           # SVM模型工厂函数
└── train_test/                # 训练测试脚本目录
    └── svm_train_test.py      # 主实验脚本

In [ ]:
def run_svm_experiment():
    print("\n=== 开始运行SVM实验 ===")
    
    from sklearn.svm import LinearSVC
    model = LinearSVC(max_iter=10000, dual=False)
    print(f"使用SVM模型: {model}")
    
    # S-FFSD数据集（已在preprocess中划分好训练集和测试集）
    print("\n--- S-FFSD数据集 ---")
    X_train, X_test, y_train, y_test = preprocess_sffsd()
    sffsd_accuracy, sffsd_auc, sffsd_f1, sffsd_ap = train_and_test(model, X_train, y_train, X_test, y_test)

    # Amazon数据集（新增划分步骤）
    print("\n--- Amazon数据集 ---")
    amazon_features, amazon_labels = preprocess_amazon()
    X_amz_train, X_amz_test, y_amz_train, y_amz_test = train_test_split(
        amazon_features, amazon_labels, test_size=0.4, random_state=42
    )
    amazon_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
    amazon_accuracy, amazon_auc, amazon_f1, amazon_ap = train_and_test(
        amazon_model, X_amz_train, y_amz_train, X_amz_test, y_amz_test
    )

    # YelpChi数据集（新增划分步骤）
    print("\n--- YelpChi数据集 ---")
    yelpchi_features, yelpchi_labels = preprocess_yelpchi()
    X_yc_train, X_yc_test, y_yc_train, y_yc_test = train_test_split(
        yelpchi_features, yelpchi_labels, test_size=0.4, random_state=42
    )
    yelpchi_model = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
    yelpchi_accuracy, yelpchi_auc, yelpchi_f1, yelpchi_ap = train_and_test(
        yelpchi_model, X_yc_train, y_yc_train, X_yc_test, y_yc_test
    )

    results = {
        'S-FFSD': {'Accuracy': sffsd_accuracy, 'AUC': sffsd_auc, 'F1': sffsd_f1, 'AP': sffsd_ap},
        'Amazon': {'Accuracy': amazon_accuracy, 'AUC': amazon_auc, 'F1': amazon_f1, 'AP': amazon_ap},
        'YelpChi': {'Accuracy': yelpchi_accuracy, 'AUC': yelpchi_auc, 'F1': yelpchi_f1, 'AP': yelpchi_ap}
    }
    
    print("\n=== 实验结果汇总 ===")
    for dataset, metrics in results.items():
        print(f"{dataset}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.4f}")
    
    return results

1. 数据划分策略
S-FFSD：基于时间顺序划分（由preprocess_sffsd内部实现）
前 60% 数据作为训练集
后 40% 数据作为测试集
Amazon/YelpChi：随机划分
train_test_split(test_size=0.4, random_state=42)
固定随机种子确保结果可复现
2. 评估指标体系
Accuracy：准确率
AUC：ROC 曲线下面积（二分类或多分类宏平均）
F1 Score：精确率与召回率的调和平均数（宏平均）
AP (Average Precision)：平均精度，衡量排序质量

In [ ]:
def preprocess_sffsd():
    file_path = os.path.join(DATADIR, 'S-FFSD.csv')
    print(f"尝试加载文件: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")
    
    data = pd.read_csv(file_path)
    print(f"S-FFSD数据形状: {data.shape}")
    
    time_column = 'Time'
    if time_column not in data.columns:
        raise ValueError(f"数据集必须包含时间列 '{time_column}'")
    
    data = data.sort_values(by=time_column, ascending=True).reset_index(drop=True)
    
    numeric_columns = data.select_dtypes(include=['number']).columns.tolist()
    categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
    
    unique_counts = {col: data[col].nunique() for col in categorical_columns}
    print("\n分类列唯一值数量:")
    for col, count in unique_counts.items():
        print(f"- {col}: {count}")
    
    # 保留Source列，仅删除其他高基数列
    high_cardinality_columns = [col for col, count in unique_counts.items() if count > 1000 and col != 'Source']
    if high_cardinality_columns:
        print(f"\n警告: 以下高基数列将被删除: {high_cardinality_columns}")
        categorical_columns = [col for col in categorical_columns if col not in high_cardinality_columns]
    else:
        print("\n提示: 无高基数列需要删除")
    
    features = []
    
    # 处理Source列（哈希编码，移除input_type参数）
    if 'Source' in data.columns:
        data['Source'] = data['Source'].astype(str)  # 确保为字符串类型
        hasher = HashingVectorizer(n_features=1000, alternate_sign=False)  # 正确参数
        source_hash = hasher.transform(data['Source'])
        features.append(source_hash)
    
    # 处理其他分类列（One-Hot编码）
    other_categorical_columns = [col for col in categorical_columns if col != 'Source']
    if other_categorical_columns:
        encoder = OneHotEncoder(sparse_output=True)
        encoded_features = encoder.fit_transform(data[other_categorical_columns])
        features.append(encoded_features)
    
    # 处理数值列
    if numeric_columns:
        numeric_sparse = csr_matrix(data[numeric_columns].values)
        features.append(numeric_sparse)
    
    features = hstack(features).tocsr() if features else csr_matrix([])
    
    # 处理标签
    labels = data['Labels']
    if labels.dtype == 'object':
        print("警告: 标签列是字符串类型，尝试转换为数值类型")
        labels = pd.to_numeric(labels, errors='coerce')
        valid_indices = labels.notna()
        features = features[valid_indices]
        labels = labels[valid_indices]
    
    # 时间划分
    total_samples = len(labels)
    train_size = int(0.6 * total_samples)
    X_train, X_test = features[:train_size], features[train_size:]
    y_train, y_test = labels[:train_size], labels[train_size:]
    
    print(f"划分后: 训练集特征={X_train.shape}, 测试集特征={X_test.shape}")
    return X_train, X_test, y_train, y_test

`preprocess_sffsd()`函数主要用于预处理S-FFSD金融欺诈数据集，首先加载数据并验证时间列存在性，按时间排序后分离数值列与分类列。针对分类列中的高基数特征（如Source），使用哈希编码将其转换为固定维度（1000维）的特征向量以避免维度爆炸，同时过滤掉其他唯一值超过1000的高基数列。对剩余分类列进行稀疏One-Hot编码，数值列转换为稀疏矩阵，最终合并所有特征形成稀疏矩阵。处理标签时将字符串类型转换为数值并过滤无效样本，最后按时间顺序划分训练集（60%）和测试集（40%），确保数据符合时间敏感型场景的建模需求，为后续SVM模型训练提供结构合理、特征高效的输入数据。

In [ ]:
def preprocess_amazon():
    """
    预处理Amazon数据集
    """
    file_path = os.path.join(DATADIR, 'Amazon.mat')
    print(f"尝试加载文件: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")
    
    amz = loadmat(file_path)
    feat_data = pd.DataFrame(amz['features'].todense().A)
    labels = pd.DataFrame(amz['label'].flatten())[0]
    print(f"Amazon数据形状: 特征={feat_data.shape}, 标签={labels.shape}")
    return feat_data, labels

def preprocess_yelpchi():
    """
    预处理YelpChi数据集
    """
    file_path = os.path.join(DATADIR, 'YelpChi.mat')
    print(f"尝试加载文件: {file_path}")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"文件不存在: {file_path}")
    
    yelp = loadmat(file_path)
    feat_data = pd.DataFrame(yelp['features'].todense().A)
    labels = pd.DataFrame(yelp['label'].flatten())[0]
    print(f"YelpChi数据形状: 特征={feat_data.shape}, 标签={labels.shape}")
    return feat_data, labels

`preprocess_amazon()`和`preprocess_yelpchi()`函数用于预处理Amazon和YelpChi数据集，两者逻辑高度相似。函数首先拼接数据文件路径并尝试加载`.mat`格式文件，若文件不存在则抛出异常。成功加载后，使用`loadmat`解析文件内容，将特征矩阵（`features`）转换为 pandas DataFrame，标签向量（`label`）经展平后提取为一维序列。过程中打印数据形状信息以便调试，最终返回特征数据和标签。该预处理流程适用于密集型特征矩阵的场景，直接保留原始特征结构，为后续随机划分训练集与测试集提供基础，适用于非时间序列的分类任务。

In [8]:
def train_and_test(model, X_train, y_train, X_test, y_test):
    """
    训练并测试模型，处理二分类和多分类情况
    """
    print(f"训练数据: 特征形状={X_train.shape}, 标签形状={y_train.shape}")
    model.fit(X_train, y_train)
    
    # 预测类别
    y_pred = model.predict(X_test)
    
    # 获取预测概率或决策函数值
    try:
        # 尝试获取预测概率（对于支持predict_proba的模型）
        y_score = model.predict_proba(X_test)
    except AttributeError:
        # 如果模型不支持predict_proba，则尝试使用decision_function
        y_score = model.decision_function(X_test)
    
    # 检查标签是否为二分类
    unique_labels = np.unique(y_train)
    is_binary = len(unique_labels) == 2
    
    # 计算评估指标
    accuracy = accuracy_score(y_test, y_pred)
    
    if is_binary:
        # 二分类情况
        if y_score.ndim > 1:
            y_score = y_score[:, 1]  # 对于二分类，只取正类的分数
        auc = roc_auc_score(y_test, y_score)
        ap = average_precision_score(y_test, y_score)
    else:
        # 多分类情况
        # 将标签转换为one-hot编码
        y_test_bin = label_binarize(y_test, classes=unique_labels)
        
        # 计算宏平均AUC
        auc = roc_auc_score(y_test_bin, y_score, multi_class='ovr', average='macro')
        
        # 计算宏平均AP
        ap = average_precision_score(y_test_bin, y_score, average='macro')
    
    f1 = f1_score(y_test, y_pred, average='macro')
    
    print(f"评估指标: Accuracy={accuracy:.4f}, AUC={auc:.4f}, F1={f1:.4f}, AP={ap:.4f}")
    return accuracy, auc, f1, ap

if __name__ == "__main__":
    try:
        print("启动SVM实验脚本...")
        print(f"数据目录: {DATADIR}")
        results = run_svm_experiment()
        print("\n实验完成!")
    except Exception as e:
        print(f"错误: {e}")
        import traceback
        traceback.print_exc()

启动SVM实验脚本...
数据目录: E:/Finance_fraud/Finance_fraud\data

=== 开始运行SVM实验 ===
使用SVM模型: LinearSVC(dual=False, max_iter=10000)

--- S-FFSD数据集 ---
尝试加载文件: E:/Finance_fraud/Finance_fraud\data\S-FFSD.csv
S-FFSD数据形状: (77881, 7)

分类列唯一值数量:
- Source: 30346
- Target: 886
- Location: 296
- Type: 166

提示: 无高基数列需要删除
划分后: 训练集特征=(46728, 2351), 测试集特征=(31153, 2351)
训练数据: 特征形状=(46728, 2351), 标签形状=(46728,)
评估指标: Accuracy=0.8983, AUC=0.8529, F1=0.6191, AP=0.7009

--- Amazon数据集 ---
尝试加载文件: E:/Finance_fraud/Finance_fraud\data\Amazon.mat
Amazon数据形状: 特征=(11944, 25), 标签=(11944,)
训练数据: 特征形状=(7166, 25), 标签形状=(7166,)
评估指标: Accuracy=0.9753, AUC=0.8813, F1=0.8921, AP=0.7553

--- YelpChi数据集 ---
尝试加载文件: E:/Finance_fraud/Finance_fraud\data\YelpChi.mat
YelpChi数据形状: 特征=(45954, 32), 标签=(45954,)
训练数据: 特征形状=(27572, 32), 标签形状=(27572,)
评估指标: Accuracy=0.8525, AUC=0.7526, F1=0.4602, AP=0.4168

=== 实验结果汇总 ===
S-FFSD:
  Accuracy: 0.8983
  AUC: 0.8529
  F1: 0.6191
  AP: 0.7009
Amazon:
  Accuracy: 0.9753
  AUC: 0.8813
  F1: 0.8921
  

`train_and_test`函数用于训练和评估SVM模型，支持二分类和多分类场景。函数首先打印训练数据形状并拟合模型，然后生成预测结果和分数（概率或决策值）。通过判断标签唯一性确定分类类型：二分类时提取正类分数计算AUC和AP，多分类则将标签二值化后计算宏平均指标。最终返回准确率、AUC、F1分数和平均精度，并打印结果。主程序入口部分负责启动实验流程，处理异常并打印结果汇总，确保实验流程的完整性和健壮性。

一、项目概述
1. 项目目标
实现基于支持向量机（SVM）的分类实验，支持多数据集预处理、模型训练与多指标评估，验证 SVM 在不同场景下的性能表现。
2. 技术栈
编程语言：Python
关键库：
scikit-learn：模型训练与评估（SVM、数据划分、指标计算）
pandas/numpy：数据处理与数值计算
scipy：稀疏矩阵操作与.mat文件加载
HashingVectorizer：高基数特征哈希编码
二、项目结构
plaintext
E:/Finance_fraud/Finance_fraud  
├── data/               
│   ├── S-FFSD.csv       
│   ├── Amazon.mat        
│   └── YelpChi.mat      
├── models/               
│   └── svm_model.py      
├── train_test/        
│   └── svm_train_test.py   
└── requirements.txt    
三、核心功能模块
1. 数据预处理模块
1.1 S-FFSD 数据集（时间序列处理）
核心逻辑：
时间排序：按Time列升序排列，确保前 60% 为训练集，后 40% 为测试集。
高基数特征处理：
过滤唯一值 > 1000 的分类列。
对Source列使用哈希编码，固定维度为 1000，避免 One-Hot 编码的维度爆炸。
特征编码：
分类列：稀疏 One-Hot 编码。
数值列：转换为稀疏矩阵（csr_matrix）。
输出：稀疏特征矩阵与数值标签，按时间划分的训练集 / 测试集。
1.2 Amazon/YelpChi 数据集（密集矩阵处理）
核心逻辑：
直接加载.mat文件，返回密集特征矩阵与标签。
使用train_test_split随机划分训练集 / 测试集（测试集占比 40%）。
2. 模型训练与评估模块
2.1 模型选择
数据集类型	            模型选择	                   原因
S-FFSD（稀疏高维）	    LinearSVC	      支持稀疏输入，适合大规模数据
Amazon/YelpChi（密集）	SGDClassifier	  支持增量学习，内存效率高
2.2 评估指标
二分类：Accuracy、AUC-ROC、F1 分数、平均精度（AP）。
多分类：宏平均 AUC-ROC、宏平均 F1 分数、宏平均 AP（通过label_binarize转换为 One-Hot 编码计算）。
3. 实验运行模块
初始化模型，依次处理 S-FFSD、Amazon、YelpChi 数据集。